https://adventofcode.com/2022/day/17

In [124]:
from collections import defaultdict
from itertools import cycle, repeat, chain
from math import lcm

In [8]:
with open("data/17.txt") as fh:
    data = fh.read().strip()

In [9]:
data[:20]

'>>><>><<<<><<<>>><><'

In [10]:
testdata = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"

In [11]:
rocks = [
    {0+1j, 1+1j, 2+1j, 3+1j},
    {1+1j, 2j, 1+2j, 2+2j, 1+3j},
    {0+1j, 1+1j, 2+1j, 2+2j, 2+3j},
    {0+1j, 0+2j, 0+3j, 0+4j},
    {0+1j, 1+1j, 0+2j, 1+2j}
]

In [1]:
def step(puff, rock, cavern):
    if puff == "<":
        trial = {p - 1 for p in rock}
        if not (any(p.real < 0 for p in trial) or cavern.intersection(trial)):
            rock = trial
    elif puff == ">":
        trial = {p + 1 for p in rock}
        if not (any(p.real > 6 for p in trial) or cavern.intersection(trial)):
            rock = trial
    trial = {p - 1j for p in rock}
    if cavern.intersection(trial) or any(p.imag < 1 for p in trial):
        return (None, cavern.union(rock))
    else:
        return (trial, cavern)

def start_rockfall(rock, cavern):
    height = max((p.imag for p in cavern), default=0)
    return {p + 2 + height * 1j + 3j for p in rock}

def sortcomplex(s):
    return sorted(s, key=lambda x: (-x.imag, x.real))

def make_puffstream(s):
    return chain.from_iterable(repeat(s))

def tetris_rocks(n, rocks, puffstream, cavern):
    rockstore = cycle(rocks)
    for _ in range(n):
        rock = start_rockfall(next(rockstore), cavern)
        while rock is not None:
            rock, cavern = step(next(puffstream), rock, cavern)
    return cavern

In [51]:
%%time
puffstream = chain.from_iterable(repeat(testdata))
testcavern = tetris_rocks(2022, rocks, puffstream, set())
max(p.imag for p in testcavern)

CPU times: user 868 ms, sys: 0 ns, total: 868 ms
Wall time: 868 ms


3068.0

Part 1

In [176]:
%%time
puffstream = make_puffstream(data)
cavern = tetris_rocks(2022, rocks, puffstream, set())
max(p.imag for p in cavern)

CPU times: user 956 ms, sys: 3.95 ms, total: 960 ms
Wall time: 977 ms


3127.0

Part 2
Find period

In [173]:
def tops(cavern):
    D = {k:0 for k in range(7)}
    for p in cavern:
        D[p.real] = max(D[p.real], p.imag)
    maxheight = max(D.values())
    return tuple(int(maxheight - D.get(i, 0)) for i in range(7))

In [174]:
def top_rocks(n, rocks, puffstream, cavern=None):
    if cavern is None:
        cavern = set()
    rockstore = cycle(rocks)
    rockcount = 0
    D = defaultdict(list)
    while True:
        rock = start_rockfall(next(rockstore), cavern)
        rockcount += 1
        if rockcount >= n:
            break
        while rock is not None:
            rock, cavern = step(next(puffstream), rock, cavern)
        D[tops(cavern)].append(rockcount)
    return D



In [175]:
%%time
puffstream = make_puffstream(testdata)
D = top_rocks(300, rocks, puffstream)
D

CPU times: user 104 ms, sys: 4.05 ms, total: 108 ms
Wall time: 107 ms


defaultdict(list,
            {(1, 1, 0, 0, 0, 0, 1): [1],
             (4, 4, 1, 0, 1, 3, 4): [2],
             (2, 2, 0, 2, 3, 5, 6): [3],
             (3, 3, 1, 3, 0, 6, 7): [4],
             (5, 5, 3, 5, 0, 0, 9): [5],
             (6, 0, 0, 0, 0, 1, 10): [6],
             (9, 1, 0, 1, 3, 4, 13): [7],
             (11, 3, 2, 2, 2, 0, 15): [8],
             (13, 5, 4, 4, 0, 2, 17): [9],
             (3, 3, 4, 4, 0, 2, 17): [10],
             (4, 4, 5, 0, 0, 0, 0): [11],
             (7, 7, 1, 0, 1, 3, 3): [12],
             (9, 9, 3, 2, 2, 2, 0): [13],
             (5, 9, 3, 2, 2, 2, 0): [14],
             (7, 11, 5, 4, 4, 0, 0): [15],
             (8, 12, 0, 0, 0, 0, 1): [16],
             (11, 15, 1, 0, 1, 3, 4): [17],
             (14, 18, 2, 2, 0, 6, 7): [18],
             (18, 22, 6, 6, 0, 10, 11): [19],
             (18, 4, 4, 6, 0, 10, 11): [20],
             (19, 0, 0, 0, 0, 11, 12): [21],
             (21, 2, 2, 2, 1, 0, 1): [22],
             (24, 5, 5, 5, 2, 2, 0): [23],


By inspection, period for testdata is 35

In [157]:
%%time
puffstream = make_puffstream(data)
D = top_rocks(10000, rocks, puffstream)
print(len(D))
D

1660
CPU times: user 1min 33s, sys: 0 ns, total: 1min 33s
Wall time: 1min 33s


defaultdict(list,
            {(1, 1, 0, 0, 0, 0, 1): [1],
             (4, 4, 1, 0, 1, 3, 4): [2],
             (2, 2, 0, 2, 3, 5, 6): [3],
             (3, 3, 1, 3, 0, 6, 7): [4],
             (4, 4, 0, 0, 1, 7, 8): [5],
             (5, 0, 0, 0, 0, 8, 9): [6],
             (8, 3, 1, 0, 1, 11, 12): [7],
             (10, 5, 3, 2, 2, 2, 0): [8,
              233,
              448,
              449,
              1123,
              1933,
              2148,
              2149,
              2823,
              3633,
              3848,
              3849,
              4523,
              5333,
              5548,
              5549,
              6223,
              7033,
              7248,
              7249,
              7923,
              8733,
              8948,
              8949,
              9623],
             (10, 1, 3, 2, 2, 2, 0): [9],
             (0, 0, 4, 3, 3, 3, 1): [10, 1765, 3465, 5165, 6865, 8565],
             (0, 0, 4, 0, 0, 0, 0): [11],
             (1, 0

Period for puzzle data is 1700

Now find out how much height grows in each period

In [133]:
def tetris_rock_period(period, n_periods, rocks, puffstream, cavern=None):
    if cavern is None:
        cavern = set()
    rockstore = cycle(rocks)
    periodcount = 0
    rockcount = 0
    while True:
        rock = start_rockfall(next(rockstore), cavern)
        rockcount += 1
        while rock is not None:
            rock, cavern = step(next(puffstream), rock, cavern)
        if not rockcount % period:
            yield(rockcount, max(p.imag for p in cavern))
            periodcount += 1
            if periodcount == n_periods:return


In [167]:
period = 35
n_periods = 10
puffstream = make_puffstream(testdata)
L = list(tetris_rock_period(period, n_periods, rocks, puffstream))
L

[(35, 60.0),
 (70, 113.0),
 (105, 166.0),
 (140, 219.0),
 (175, 272.0),
 (210, 325.0),
 (245, 378.0),
 (280, 431.0),
 (315, 484.0),
 (350, 537.0)]

Testdata height grows 53 per 35-rock period.

In [169]:
%%time
period = 1700
n_periods = 5
puffstream = make_puffstream(data)
L = list(tetris_rock_period(period, n_periods, rocks, puffstream))
L

CPU times: user 14 s, sys: 0 ns, total: 14 s
Wall time: 14.1 s


[(1700, 2630.0),
 (3400, 5253.0),
 (5100, 7876.0),
 (6800, 10499.0),
 (8500, 13122.0)]

In [171]:
10499 - 7876

2623

Puzzle data height grows 2623 per 1700 rock period.

In [136]:
trillion = 1_000_000_000_000

In [138]:
divmod(trillion, 35)

(28571428571, 15)

In [144]:
%%time
puffstream = make_puffstream(testdata)
cavern = tetris_rocks(15, rocks, puffstream, set())
print(max(p.imag for p in cavern))

25.0
CPU times: user 133 ms, sys: 0 ns, total: 133 ms
Wall time: 131 ms


In [145]:
25 + trillion // 35 * 53

1514285714288

^ Test height after 1 trillion rocks

In [163]:
divmod(trillion, 1700)

(588235294, 200)

In [164]:
%%time
puffstream = make_puffstream(data)
cavern = tetris_rocks(200, rocks, puffstream, set())
print(max(p.imag for p in cavern))

318.0
CPU times: user 16.1 ms, sys: 37 µs, total: 16.1 ms
Wall time: 15.1 ms


In [172]:
318 + trillion // 1700 * 2623

1542941176480

^ Part 2, height after 1 trillion rocks